In [1]:
import borg

Initializing console.
[INFO   ] libLSS version ed62e01f04bce073170912a9ab130cd24e37b89e built-in modules 
[INFO S ] Registered forward models:
[INFO S ]    2LPT_CIC, 2LPT_CIC_OPENMP, 2LPT_DOUBLE, 2LPT_NGP, ALTAIR_AP, Downgrade
[INFO S ]    EnforceMass, HADES_LOG, HADES_PT, Haar, LPT_CIC, LPT_CIC_OPENMP, LPT_DOUBLE
[INFO S ]    LPT_NGP, LPT_NU_CIC, LPT_NU_CIC_OPENMP, PATCH_MODEL, PM_CIC, PM_CIC_OPENMP
[INFO S ]    PM_CIC_OPENMP2, PRIMORDIAL, PRIMORDIAL_AS, PRIMORDIAL_FNL, QLPT, QLPT_RSD
[INFO S ]    Softplus, TRANSFER_CLASS, TRANSFER_EHU, Transfer, Upgrade, bias::BrokenPowerLaw
[INFO S ]    bias::DoubleBrokenPowerLaw, bias::EFT, bias::EFT_Thresh, bias::Linear
[INFO S ]    bias::ManyPower_1^1, bias::ManyPower_1^2, bias::ManyPower_1^4, bias::ManyPower_2^2
[INFO S ]    bias::Noop, bias::PowerLaw
[INFO S ] Registered forward models (v3):
[INFO S ]    KappaSphere
[INFO S ] Registered likelihoods:
[INFO S ]    BORG_NULL, BORG_POISSON, BORG_REDUCED_SHEAR, BORG_SHEAR, BORG_STUDENT_T
[INFO S ]  

In [ ]:
import borg
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class Genet(borg.forward.BaseForwardModel):
   # Constructor
   def __init__(self, box):
     super().__init__(box, box)

   # IO "preferences"
   def getPreferredInput(self):
     return borg.forward.PREFERRED_REAL

   def getPreferredOutput(self):
     return borg.forward.PREFERRED_REAL

   # Forward part

   def forwardModel_v2(self, input_array):
     self.save = jax.numpy.array(input_array)

   def getDensityFinal(self, output_array):
     output_array[:] = self.save**2

   # Adjoint part

   def adjointModel_v2(self, input_ag):
     self.ag = input_ag

   def getAdjointModel(self, output_ag):
     output_ag[:] = 2 * self.ag * self.save

In [ ]:
global step_id
chain = 0

# setup the box
bb = borg.forward.BoxModel()

L = 250
Nt = 128

bb.L = L,L,L
bb.N = Nt,Nt,Nt

# Initialize some default cosmology
cosmo = borg.cosmo.CosmologicalParameters()

# Fiducial scale factor to express initial conditions
z_start = 69
a0 = 0.001
nsteps = 20

chain = borg.forward.ChainForwardModel(bb)
# Add primordial fluctuations
chain.addModel(borg.forward.models.Primordial(bb, a0))
# Add E&Hu transfer function
chain.addModel(borg.forward.models.EisensteinHu(bb))
# Run an LPT model from a=0.0 to af. The ai=a0 is the scale factor at which the IC are expressed
#lpt = borg.forward.models.BorgLpt(bb, bb, ai=a0, af=1.0)
lpt = borg.forward.models.BorgPM(bb, bb, ai=a0, af=1.0, z_start=z_start, particle_factor=1, force_factor=2, supersampling=1, nsteps=nsteps, tCOLA = 1)
chain.addModel(lpt)

genet = Genet()
chain.addModel(genet)

pos_array = np.zeros((nsteps,Nt**3,3))
step_id=0
def notif(t, Np, ids, poss, vels):
    global step_id
    print(f"Step {t} / {step_id}  (Np={Np})")
    pos_array[step_id,:,:] = poss
    step_id+=1

lpt.setStepNotifier(notif, with_particles=True)

# Set the cosmology
chain.setCosmoParams(cosmo)

# Generate white noise: it has to be scaled by 1/N**(3./2) to be one in Fourier
np.random.seed(42)
s_field = np.random.normal(0,1,(Nt,Nt,Nt))
ic = np.fft.rfftn(s_field/np.sqrt(Nt**3))
print('np.std(ic) = ',np.std(ic))
#ic = np.fft.rfftn(np.random.randn(Ng, Ng, Ng)/np.sqrt(Ng**3))
delta_m = np.zeros((Nt,Nt,Nt))

# RUN!
chain.forwardModel_v2(ic)
chain.getDensityFinal(delta_m)

# Get pos
abs_pos_COLA = np.zeros((lpt.getNumberOfParticles(),3))
lpt.getParticlePositions(abs_pos_COLA)